This notebook loads and displays the results of the GEE analyses.

In [1]:
# Add the parent directory of the current working directory to the Python path at runtime. 
# In order to import modules from the src directory.
import os
import sys 

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [2]:
import glob
import pickle
from prettytable import PrettyTable
from src.stat_utils import *

results_dir = os.path.join(parent_dir, 'results/statistics')
full_model_file = os.path.join(results_dir, 'gee_full.pkl')

In [3]:
def add_row_to_table(session, results, table):
    predictors = results.model.exog_names
    for i, predictor in enumerate(predictors):
        coefficient = results.params[predictor]
        confidence_interval = [results.conf_int()[0][predictor], results.conf_int()[1][predictor]]
        standard_error = results.standard_errors()[i]
        wald = results.wald_test(predictor, scalar=True)
        chi = wald.statistic
        pvalue = wald.pvalue
        table.add_row([session, predictor, coefficient, confidence_interval, standard_error, chi, pvalue])

    return table

### Results of full model

In [4]:
with open(full_model_file, 'rb') as f:
    gee_full = pickle.load(f)

print(gee_full.summary())

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                48000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                6000
                      Estimating Equations   Max. cluster size:                6000
Family:                           Binomial   Mean cluster size:              6000.0
Dependence structure:         Exchangeable   Num. iterations:                    56
Date:                     Tue, 19 Mar 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         14:36:23
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           1.6459  

In [5]:
print_wald_chi_square(gee_full)

Wald Chi-Square:
+---------------------------------+--------------------+------------------------+
|             Variable            |     Chi-Square     |        p-value         |
+---------------------------------+--------------------+------------------------+
|            Intercept            | 10.008055874170422 | 0.0015585696172313117  |
|      ContrastHeterogeneity      | 49.15577499337458  | 2.364213012996858e-12  |
|          GridCoarseness         | 58.308691973120965 | 2.2405200226713208e-14 |
|            SessionID            | 6.461520831356995  |  0.011023525757624731  |
| SessionID:ContrastHeterogeneity | 51.35228258690842  | 7.719219789482875e-13  |
|     SessionID:GridCoarseness    | 0.8266047296477383 |   0.3632561477783306   |
+---------------------------------+--------------------+------------------------+


### Simple effects for significant interactions

In [6]:
table = PrettyTable()
table.field_names = ['session', 'predictor', 'beta-coefficient', '95% confidence interval', 'sdt error', 'Wald chi-square', 'P-value']

files = glob.glob(os.path.join(results_dir, 'gee_*.pkl'))
files.remove(full_model_file)
files = sorted(files)

for i, file in enumerate(sorted(files)):
    with open(file, 'rb') as f:
        results = pickle.load(f)
    table = add_row_to_table(i + 1, results, table)


print(table)

+---------+-----------------------+---------------------+--------------------------------------------+---------------------+--------------------+------------------------+
| session |       predictor       |   beta-coefficient  |          95% confidence interval           |      sdt error      |  Wald chi-square   |        P-value         |
+---------+-----------------------+---------------------+--------------------------------------------+---------------------+--------------------+------------------------+
|    1    |       Intercept       |  1.3740677599747717 |  [0.8504068762125485, 1.897728643736995]   |  0.2671788297605433 | 26.449180045587497 | 2.7056330342769475e-07 |
|    1    | ContrastHeterogeneity |  -1.458433509386122 |  [-2.01937364915105, -0.8974933696211937]  |  0.2861992078372626 | 25.967875108474946 | 3.4714616159320283e-07 |
|    2    |       Intercept       |  1.975328629585899  |  [1.3820671746272293, 2.5685900845445686]  | 0.30268997779461276 | 42.58754775033208  |